In [15]:
from aocutils.grid import arr_to_dict, arr_neighbors

lines = open('input.txt').read().splitlines()
lines = [list(map(int, list(line))) for line in lines]

# finally some library functions came in handy!
oct = arr_to_dict(lines)
neighbors = arr_neighbors(lines, diag=True)

flashed = 0
for step in range(1, 10000000000000):
    for octo in oct:
        oct[octo] += 1
    
    flashing = {octo for octo in oct if oct[octo] > 9}
    while flashing:
        f = flashing.pop()
        flashed += 1
        oct[f] = 0
        for neigh in neighbors[f]:
            if oct[neigh] != 0:
                oct[neigh] += 1
                if oct[neigh]>9:
                    flashing.add(neigh)

    if step == 100:
        print(f'part 1, {flashed} octopuses flashed')
    if all(v==0 for v in oct.values()):
        print(f'part 2, synchronous flashing after {step} steps (and {flashed} flashes')
        break

part 1, 1652 octopuses flashed
part 2, synchronous flashing after 220 steps (and 3480 flashes


In [2]:
# alternative with list of lists instead of dicts. feels clunckier though. I do like my own iterate
# also there are basically two ways: make the recursion function 'increase_light', or make it 'flash'
from aocutils.grid import arr_neighbors, iterate

lines = open('input.txt').read().splitlines()
oct = [list(map(int, list(line))) for line in lines]

# finally some library functions came in handy!
neighbors = arr_neighbors(oct, diag=True)

def flash(r,c):
    # used globals because why not
    global flashed
    flashed += 1
    oct[r][c] = 0
    for rr,cc in neighbors[(r,c)]:
        val = oct[rr][cc]
        if val != 0 and val != 10:
            oct[rr][cc] = val + 1
        if val == 9:
            flash(rr,cc)

flashed = 0

for step in range(1, 10000000000000):
    for r,c, val in iterate(oct):
        oct[r][c] += 1
    for r,c, val in iterate(oct):
        if val == 10:
            flash(r,c)
       
    if step == 100:
        print(f'part 1, {flashed} octopuses flashed')

    if all(val==0 for i,j,val in iterate(oct)):
        print(f'part 2, synchronous flashing after {step} steps (and {flashed} flashes')
        break

part 1, 1652 octopuses flashed
part 2, synchronous flashing after 220 steps (and 3480 flashes


In [20]:
# can always be better, from https://www.reddit.com/user/4HbQ/
import numpy as np
from scipy.signal import convolve

x = np.genfromtxt(open('input.txt'), delimiter=1)
fp = [[1,1,1],[1,0,1],[1,1,1]]
count = 0

for i in range(1, 3):
    x += 1
    x_ = x
    while True:
        flash = x > 9
        x = x_ + convolve(flash, fp, mode='same')
        print(x)
        if (flash == (x > 9)).all():  # no change
            count += flash.sum()
            x[flash] = 0
            break

    if i == 100:
        print(count)
    if flash.all():
        print(i); break

[[5. 6. 3. 6. 5. 4. 7. 5. 2. 8.]
 [2. 9. 6. 2. 3. 5. 3. 6. 6. 4.]
 [6. 5. 3. 2. 5. 4. 6. 6. 3. 2.]
 [9. 5. 4. 2. 4. 3. 6. 5. 5. 8.]
 [5. 6. 2. 8. 5. 4. 9. 4. 4. 3.]
 [4. 6. 3. 2. 3. 7. 3. 2. 2. 2.]
 [4. 4. 4. 2. 6. 5. 2. 8. 4. 5.]
 [5. 4. 6. 2. 9. 4. 7. 7. 5. 2.]
 [3. 8. 6. 4. 9. 9. 2. 5. 5. 3.]
 [8. 8. 2. 8. 7. 2. 7. 9. 7. 4.]]
[[ 7.  8.  5.  7.  6.  5.  8.  6.  3.  9.]
 [ 4. 10.  8.  3.  4.  6.  4.  7.  7.  5.]
 [ 9.  8.  5.  3.  6.  5.  7.  7.  4.  3.]
 [10.  7.  5.  3.  5.  5.  8.  7.  6.  9.]
 [ 7.  8.  3.  9.  6.  6. 10.  6.  5.  4.]
 [ 5.  7.  4.  3.  4.  9.  5.  4.  3.  3.]
 [ 5.  5.  5.  4.  8.  7.  3.  9.  5.  6.]
 [ 6.  5.  7.  5. 12.  8.  9.  8.  6.  3.]
 [ 4.  9.  7.  7. 12. 12.  5.  7.  7.  4.]
 [ 9.  9.  3. 10. 10.  5. 10. 10.  9.  5.]]
[[ 7.  8.  5.  7.  6.  5.  8.  6.  3.  9.]
 [ 4. 10.  8.  3.  4.  6.  4.  7.  7.  5.]
 [ 9.  8.  5.  3.  6.  5.  7.  7.  4.  3.]
 [10.  7.  5.  3.  5.  5.  8.  7.  6.  9.]
 [ 7.  8.  3.  9.  6.  6. 10.  6.  5.  4.]
 [ 5.  7.  4.  3.  4.  

In [5]:
e = {(x,y):int(e) for x,l in enumerate(open('input.txt'))
                  for y,e in enumerate(l.strip())}

def neighbours(x,y): return [i for i in [(x+1,y+1),(x+1,y),(x+1,y-1),(x,y+1),
             (x-1,y-1),(x-1,y),(x-1,y+1),(x,y-1)] if i in e and e[i]!=0]

count = 0
for step in range(1, 1000):
    for i in e: e[i] += 1
    flashing = {i for i in e if e[i] > 9}

    while flashing:
        f = flashing.pop()
        e[f] = 0
        count += 1
        for n in neighbours(*f):
            e[n] += 1
            if e[n] > 9: flashing.add(n)

    if step == 100: print(count)
    if sum(e.values()) == 0: print(step); break

1652
220


In [36]:
x = np.arange(25).reshape(5,5)
fp = [[1,1,1],[1,1,1],[1,1,1]]
poi = (x == 0) ^ (x == 1)
poi

array([[ True,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False]])

In [37]:
convolve(poi,fp, 'same')

array([[2, 2, 1, 0, 0],
       [2, 2, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [17]:
x[0,0]=0

In [18]:
x.all()

False